In [ ]:
import os
import numpy as np
import rasterio
import geopandas as gpd
from rasterio.mask import mask
from rasterio.warp import reproject, Resampling
from glob import glob

# === CONFIG ===
root_dir = "/datasets/work/ev-veg-height/work/Tasks/Carbon_stock/AUG/Lorraine_station/NDVI_LUT/Sentinel_ndvi/"
output_dir = "/datasets/work/ev-veg-height/work/Tasks/Carbon_stock/AUG/Lorraine_station/NDVI_LUT/Sentinel_lut/"
input_folder = os.path.join(root_dir, "2021")
output_file = os.path.join(output_dir, "2021_ht_cal.tif")
sd_output_file = os.path.join(output_dir, "2021_sd.tif")
mean_output_file = os.path.join(output_dir, "2021_mean.tif")

input_shapefile = "/datasets/work/ev-veg-height/work/Tasks/Carbon_stock/AUG/Lorraine_station/Lorraine_station_outer_boundary.shp"
veg_height_file = "/datasets/work/ev-veg-height/work/Tasks/Carbon_stock/AUG/Lorraine_station/NDVI_LUT/height/ht_2021.tif"  # your vegetation height raster

nodata_value_default = -3000.0
default_class = 60   # Non-agriculture


# === CLASS RULES ===
class_rules = [
    (10, 0.0, 0.01, 0.4, lambda h: h > 3000),                # Forest
    (20, 0.0, 0.1, 0.4, lambda h: (h > 500) & (h <= 3000)),     # Woodland
    (30, 0.0, 0.12, 0.35, lambda h: (h > 200) & (h <= 500)),    # Shrubland
    (40, 0.0, np.inf, 0.25, lambda h: (h <= 200)),                # Grassland
]

# === FUNCTIONS ===
def reproject_raster(input_path, dst_crs, dst_transform, dst_width, dst_height, resampling=Resampling.nearest):
    """Reproject raster to match NDVI grid."""
    with rasterio.open(input_path) as src:
        src_data = src.read(1)
        dst_data = np.empty((dst_height, dst_width), dtype=src.meta['dtype'])
        reproject(
            source=src_data,
            destination=dst_data,
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=dst_transform,
            dst_crs=dst_crs,
            resampling=resampling
        )
    return dst_data

def save_raster(output_path, data, template_file, transform, dtype, nodata_val=np.nan, compress="lzw"):
    """Save raster with same georeferencing as template."""
    with rasterio.open(template_file) as src:
        profile = src.profile.copy()
        profile.update({
            "height": data.shape[0],
            "width": data.shape[1],
            "transform": transform,
            "count": 1,
            "dtype": dtype,
            "compress": compress,
            "nodata": nodata_val
        })
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(data, 1)

# === LOAD SHAPEFILE ===
with rasterio.open(glob(os.path.join(input_folder, "*.tif"))[0]) as ref:
    raster_crs = ref.crs

print("Loading and reprojecting shapefile...")
gdf = gpd.read_file(input_shapefile).to_crs(raster_crs)
geometry = [g.__geo_interface__ for g in gdf.geometry]

# === READ, MASK, STACK NDVI RASTERS ===
file_list = sorted(glob(os.path.join(input_folder, "*.tif")))
stack = []

print("Masking and loading NDVI rasters...")
for file in file_list:
    with rasterio.open(file) as src:
        nodata_value = src.nodata if src.nodata is not None else nodata_value_default
        masked_data, masked_transform = mask(src, geometry, crop=True)
        data = masked_data[0].astype(np.float32)
        data[(data == nodata_value) | (data < 0)] = np.nan
        data = data + 0.15  # offset adjustment
        stack.append(data)

stack_array = np.stack(stack, axis=0)  # shape: (time, rows, cols)

# === CALCULATE NDVI STATS ===
print("Calculating NDVI mean and standard deviation...")
ndvi_mean = np.nanmean(stack_array, axis=0)
ndvi_sd = np.nanstd(stack_array, axis=0)

# === LOAD & REPROJECT VEGETATION HEIGHT ===
print("Loading and reprojecting vegetation height raster...")
veg_height = reproject_raster(
    veg_height_file,
    raster_crs,
    masked_transform,
    ndvi_mean.shape[1],  # width
    ndvi_mean.shape[0],  # height
    resampling=Resampling.bilinear
)

# Mask invalid values in veg height
veg_height = veg_height.astype(np.float32)
veg_height[(veg_height < 0) | (veg_height == 255)] = np.nan

# === CLASSIFICATION ===
print("Classifying pixels based on NDVI + height rules...")
classified = np.full(ndvi_sd.shape, default_class, dtype=np.float32)

for class_value, lower_sd, upper_sd, min_mean, height_cond in class_rules:
    condition = (
        (ndvi_sd >= lower_sd) &
        (ndvi_sd < upper_sd) &
        (ndvi_mean > min_mean) &
        height_cond(veg_height) 
        
    )
    classified[condition] = class_value

# Ensure all invalid are NaN
invalid_mask = (
    np.isnan(ndvi_sd) |
    np.isnan(ndvi_mean) |
    np.isnan(veg_height)
)
classified[invalid_mask] = np.nan

# === SAVE OUTPUTS ===
print("Saving classified, SD, and mean rasters...")
save_raster(output_file, classified, file_list[0], masked_transform, rasterio.float32, nodata_val=np.nan)
#save_raster(sd_output_file, ndvi_sd, file_list[0], masked_transform, rasterio.float32, nodata_val=np.nan)
#save_raster(mean_output_file, ndvi_mean, file_list[0], masked_transform, rasterio.float32, nodata_val=np.nan)

print(f"\n✅ Done.\nClassified: {output_file}\nSD: {sd_output_file}\nMean: {mean_output_file}")


In [ ]:
import os
import numpy as np
import rasterio
import geopandas as gpd
from rasterio.mask import mask
from rasterio.warp import reproject, Resampling
from glob import glob

# === CONFIG ===
root_dir = "/datasets/work/ev-veg-height/work/Tasks/Carbon_stock/AUG/Lorraine_station/NDVI_LUT/Sentinel_ndvi/"
output_dir = "/datasets/work/ev-veg-height/work/Tasks/Carbon_stock/AUG/Lorraine_station/NDVI_LUT/Sentinel_lut/"
input_folder = os.path.join(root_dir, "2022")
output_file = os.path.join(output_dir, "2022_lut.tif")
sd_output_file = os.path.join(output_dir, "2022_sd.tif")
mean_output_file = os.path.join(output_dir, "2022_mean.tif")

input_shapefile = "/datasets/work/ev-veg-height/work/Tasks/Carbon_stock/AUG/Lorraine_station/Lorraine_station_outer_boundary.shp"
veg_height_file = "/datasets/work/ev-veg-height/work/Tasks/Carbon_stock/AUG/Lorraine_station/NDVI_LUT/height/ht_2022.tif"  # your vegetation height raster

nodata_value_default = -3000.0
default_class = 60   # Non-agriculture

# === CLASSIFICATION RULES ===

#2021
"""class_rules = [
    (10, 0.0, 0.01, 0.4, lambda h: h > 3000),                # Forest
    (20, 0.0, 0.1, 0.4, lambda h: (h > 500)),    # Woodland
    (30, 0.0, 0.12, 0.35, lambda h: (h > 200)),   # Shrubland
    (40, 0.0, np.inf, 0.25, lambda h: True),               # Grassland
]"""

#2022
class_rules = [
    (10, 0.0, 0.01, 0.4, lambda h: h > 3000),                # Forest
    (20, 0.0, 0.1, 0.25, lambda h: (h > 500)),    # Woodland
    (30, 0.0, 0.12, 0.24, lambda h: (h > 200)),   # Shrubland
    (40, 0.0, np.inf, 0.1, lambda h: True),               # Grassland
]


def reproject_raster_to_ref(input_path, ref_crs, ref_transform, ref_width, ref_height, resampling=Resampling.bilinear):
    """Reproject and resample input raster to reference grid (CRS, transform, shape)."""
    with rasterio.open(input_path) as src:
        src_data = src.read(1)
        dst_data = np.empty((ref_height, ref_width), dtype=np.float32)
        reproject(
            source=src_data,
            destination=dst_data,
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=ref_transform,
            dst_crs=ref_crs,
            resampling=resampling,
            num_threads=2,
        )
    return dst_data

def save_raster(output_path, data, template_file, transform, dtype, nodata_val=np.nan, compress="lzw"):
    with rasterio.open(template_file) as src:
        profile = src.profile.copy()
        profile.update({
            "height": data.shape[0],
            "width": data.shape[1],
            "transform": transform,
            "count": 1,
            "dtype": dtype,
            "compress": compress,
            "nodata": nodata_val
        })
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(data, 1)

# === LOAD SHAPEFILE AND REFERENCE NDVI ===
ndvi_files = sorted(glob(os.path.join(input_folder, "*.tif")))
with rasterio.open(ndvi_files[0]) as ref:
    raster_crs = ref.crs

print("Loading and reprojecting shapefile...")
gdf = gpd.read_file(input_shapefile).to_crs(raster_crs)
geometry = [geom.__geo_interface__ for geom in gdf.geometry]

# === LOAD AND MASK NDVI STACK ===
stack = []
print("Masking and loading NDVI rasters...")
for ndvi_file in ndvi_files:
    with rasterio.open(ndvi_file) as src:
        nodata_val = src.nodata if src.nodata is not None else nodata_value_default
        masked_data, masked_transform = mask(src, geometry, crop=True)
        data = masked_data[0].astype(np.float32)
        data[(data == nodata_val) | (data < 0)] = np.nan
        stack.append(data)

stack_array = np.stack(stack, axis=0)

# Calculate NDVI stats
print("Calculating NDVI mean and standard deviation...")
ndvi_mean = np.nanmean(stack_array, axis=0)
ndvi_sd = np.nanstd(stack_array, axis=0)

# === REPROJECT VEG HEIGHT TO NDVI GRID ===
print("Reprojecting vegetation height raster to NDVI grid (matching resolution and extent)...")
veg_height = reproject_raster_to_ref(
    veg_height_file,
    raster_crs,
    masked_transform,
    ndvi_mean.shape[1],  # width
    ndvi_mean.shape[0],  # height
    resampling=Resampling.bilinear
)

veg_height = veg_height.astype(np.float32)
veg_height[(veg_height < 0) | (veg_height == 255)] = np.nan

# === CLASSIFICATION ===
print("Classifying pixels based on NDVI stats and vegetation height...")

classified = np.full(ndvi_sd.shape, default_class, dtype=np.float32)

for class_value, lower_sd, upper_sd, min_mean, height_cond in class_rules:
    condition = (
        (ndvi_sd >= lower_sd) &
        (ndvi_sd < upper_sd) &
        (ndvi_mean > min_mean) &
        height_cond(veg_height) &
        (classified == default_class)
    )
    classified[condition] = class_value



# Debug: Check if any grassland pixels violate NDVI threshold
grassland_pixels = (classified == 40)
violating_pixels = np.sum(ndvi_mean[grassland_pixels] <= 0.25)
print(f"Pixels classified as grassland with mean NDVI <= 0.25: {violating_pixels}")

# Correct any misclassified grassland pixels back to class 60
misclassified_mask = grassland_pixels & (ndvi_mean <= 0.25)
classified[misclassified_mask] = default_class


# Set nodata where ANY input raster has NaN
invalid_mask = (
    np.isnan(ndvi_sd) |
    np.isnan(ndvi_mean) |
    np.isnan(veg_height)
)
classified[invalid_mask] = np.nan

# === SAVE OUTPUTS ===
print("Saving classified, SD, and mean rasters...")
save_raster(output_file, classified, ndvi_files[0], masked_transform, rasterio.float32, nodata_val=np.nan)
#save_raster(sd_output_file, ndvi_sd, ndvi_files[0], masked_transform, rasterio.float32, nodata_val=np.nan)
#save_raster(mean_output_file, ndvi_mean, ndvi_files[0], masked_transform, rasterio.float32, nodata_val=np.nan)

print(f"\n✅ Done.\nClassified: {output_file}\nSD: {sd_output_file}\nMean: {mean_output_file}")

In [ ]:
import os
import numpy as np
import rasterio
import geopandas as gpd
from rasterio.mask import mask
from rasterio.warp import reproject, Resampling
from glob import glob

# === CONFIG ===
root_dir = "/datasets/work/ev-veg-height/work/Tasks/Carbon_stock/AUG/Lorraine_station/NDVI_LUT/Sentinel_ndvi/"
output_dir = "/datasets/work/ev-veg-height/work/Tasks/Carbon_stock/AUG/Lorraine_station/NDVI_LUT/Sentinel_lut/"
input_folder = os.path.join(root_dir, "2024")
output_file = os.path.join(output_dir, "2024_lut.tif")
sd_output_file = os.path.join(output_dir, "2024_sd.tif")
mean_output_file = os.path.join(output_dir, "2024_mean.tif")

input_shapefile = "/datasets/work/ev-veg-height/work/Tasks/Carbon_stock/AUG/Lorraine_station/Lorraine_station_outer_boundary.shp"
veg_height_file = "/datasets/work/ev-veg-height/work/Tasks/Carbon_stock/AUG/Lorraine_station/NDVI_LUT/height/ht_2020.tif"  # your vegetation height raster

nodata_value_default = -3000.0
default_class = 60   # Non-agriculture
nodata_class = 255              # NoData output value

# Classification rules: (class_value, SD_min, SD_max, NDVI_mean_min, height_condition_function)

#2021
"""class_rules = [
    (10, 0.0, 0.01, 0.4, lambda h: h > 3000),                # Forest
    (20, 0.0, 0.08, 0.38, lambda h: (h > 500)),    # Woodland
    (30, 0.0, 0.1, 0.29, lambda h: (h > 200)),   # Shrubland
    (40, 0.0, np.inf, 0.25, lambda h: (h>=0)),               # Grassland
]"""

#2022
"""class_rules = [
    (10, 0.0, 0.01, 0.4, lambda h: h > 3000),                # Forest
    (20, 0.0, 0.08, 0.24, lambda h: (h > 500)),    # Woodland
    (30, 0.0, 0.1, 0.23, lambda h: (h > 200)),   # Shrubland
    (40, 0.0, np.inf, 0.15, lambda h: (h>=0)),               # Grassland
]"""

#2023
"""class_rules = [
    (10, 0.0, 0.01, 0.4, lambda h: h > 3000),                # Forest
    (20, 0.0, 0.08, 0.21, lambda h: (h > 500)),    # Woodland
    (30, 0.0, 0.1, 0.17, lambda h: (h > 200)),   # Shrubland
    (40, 0.0, np.inf, 0.11, lambda h: (h>=0)),               # Grassland
]"""

#2024
class_rules = [
    (10, 0.0, 0.01, 0.4, lambda h: h > 3000),                # Forest
    (20, 0.0, 0.08, 0.24, lambda h: (h > 500)),    # Woodland
    (30, 0.0, 0.1, 0.22, lambda h: (h > 200)),   # Shrubland
    (40, 0.0, np.inf, 0.15, lambda h: (h>=0)),               # Grassland
]

# === FUNCTIONS ===

def save_raster(output_path, data, template_file, transform, dtype, compress="lzw"):
    with rasterio.open(template_file) as src:
        profile = src.profile.copy()
        profile.update({
            "height": data.shape[0],
            "width": data.shape[1],
            "transform": transform,
            "count": 1,
            "dtype": dtype,
            "compress": compress,
            "nodata": None  # We'll use np.nan internally and encode as 255 on output
        })
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(data, 1)

def reproject_raster_to_ref(input_file, dst_crs, dst_transform, dst_width, dst_height, nodata=None):
    with rasterio.open(input_file) as src:
        height_data = src.read(1).astype(np.float32)
        if nodata is None:
            nodata = src.nodata
        if nodata is not None:
            height_data = np.where(height_data == nodata, np.nan, height_data)

        print("Vegetation height raster stats BEFORE resampling:")
        print(f"  min: {np.nanmin(height_data)}")
        print(f"  max: {np.nanmax(height_data)}")
        print(f"  zeros: {(height_data == 0).sum()}")
        print(f"  nan count: {np.isnan(height_data).sum()}")

        dst_array = np.full((dst_height, dst_width), np.nan, dtype=np.float32)

        reproject(
            source=height_data,
            destination=dst_array,
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=dst_transform,
            dst_crs=dst_crs,
            resampling=Resampling.nearest,
            src_nodata=np.nan,
            dst_nodata=np.nan,
        )

        print("Vegetation height raster stats AFTER resampling:")
        print(f"  min: {np.nanmin(dst_array)}")
        print(f"  max: {np.nanmax(dst_array)}")
        print(f"  zeros: {(dst_array == 0).sum()}")
        print(f"  nan count: {np.isnan(dst_array).sum()}")

        return dst_array

# === MAIN PROCESS ===

# Load NDVI raster to get CRS, transform, dimensions
ndvi_files = sorted(glob(os.path.join(input_folder, "*.tif")))
with rasterio.open(ndvi_files[0]) as ref:
    raster_crs = ref.crs

print("Loading and reprojecting shapefile...")
gdf = gpd.read_file(input_shapefile).to_crs(raster_crs)
geometry = [g.__geo_interface__ for g in gdf.geometry]

# Read, mask, and stack NDVI rasters
stack = []
print("Masking and loading NDVI rasters...")
for file in ndvi_files:
    with rasterio.open(file) as src:
        nodata_value = src.nodata if src.nodata is not None else nodata_value_default
        masked_data, masked_transform = mask(src, geometry, crop=True)
        data = masked_data[0].astype(np.float32)
        data[(data == nodata_value) | (data < 0)] = np.nan
        stack.append(data)

stack_array = np.stack(stack, axis=0)  # (time, rows, cols)

# Calculate NDVI mean and standard deviation
print("Calculating NDVI mean and std dev...")
ndvi_mean = np.nanmean(stack_array, axis=0)
ndvi_sd = np.nanstd(stack_array, axis=0)

# Correct order: width = cols, height = rows
height = masked_data.shape[1]  # rows
width = masked_data.shape[2]   # cols

print("Loading and reprojecting vegetation height raster...")
veg_height = reproject_raster_to_ref(
    veg_height_file,
    raster_crs,
    masked_transform,
    width,   # width (cols)
    height   # height (rows)
)

# Mask invalid veg height values
veg_height = np.where((veg_height < 0) | np.isnan(veg_height), np.nan, veg_height)

# Initialize classified array with default class (60)
classified = np.full(ndvi_sd.shape, default_class, dtype=np.float32)

print("Classifying pixels based on SD, NDVI mean, and vegetation height...")
for class_value, lower_sd, upper_sd, min_mean, height_cond in class_rules:
    condition = (
        (ndvi_sd >= lower_sd) &
        (ndvi_sd < upper_sd) &
        (ndvi_mean > min_mean) &
        height_cond(veg_height) &
        (classified == default_class)  # only unclassified pixels
    )
    classified[condition] = class_value

# Set nodata for pixels invalid in any input
invalid_mask = (
    np.isnan(ndvi_sd) |
    np.isnan(ndvi_mean) |
    np.isnan(veg_height)
)
classified[invalid_mask] = np.nan

# Prepare output by replacing np.nan with nodata_class (255)
classified_out = classified.copy()
classified_out[np.isnan(classified_out)] = nodata_class

# Verify all arrays have the same shape before saving
print(f"NDVI mean shape: {ndvi_mean.shape}")
print(f"NDVI sd shape: {ndvi_sd.shape}")
print(f"Veg height shape: {veg_height.shape}")
print(f"Classified shape: {classified.shape}")

print("Saving classified raster...")
save_raster(output_file, classified_out.astype(np.uint8), ndvi_files[0], masked_transform, rasterio.uint8)

print("Saving NDVI std dev raster...")
save_raster(sd_output_file, np.nan_to_num(ndvi_sd, nan=-9999), ndvi_files[0], masked_transform, rasterio.float32)

print("Saving NDVI mean raster...")
save_raster(mean_output_file, np.nan_to_num(ndvi_mean, nan=-9999), ndvi_files[0], masked_transform, rasterio.float32)

print(f"\n✅ Done.\nClassified: {output_file}\nSD: {sd_output_file}\nMean: {mean_output_file}")